# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [6]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd
import math

df = pd.read_csv("strawberry-prices.tsv", sep="\t")
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)
df["moy"] = df["month"].dt.month  # month-of-year
df.head()

train = df[df["month"].dt.year <= 2023].copy()
test_2024 = df[df["month"].dt.year == 2024].copy()

seasonal_mean_train = train.groupby("moy")["price"].mean()

pred_2024 = test_2024["moy"].map(seasonal_mean_train).to_numpy()

backtest = pd.DataFrame({
    "month": test_2024["month"].dt.strftime("%Y-%m-01"),
    "price": np.round(pred_2024, 6)
})

backtest.to_csv("strawberry-backtest.tsv", sep="\t", index=False)
backtest.head()

,month,price
48,2024-01-01,4.36275
49,2024-02-01,4.09125
50,2024-03-01,3.68875
51,2024-04-01,3.78720
52,2024-05-01,3.52625


Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [7]:
# YOUR CHANGES HERE

residuals = test_2024["price"].to_numpy() - pred_2024

mean_resid = float(np.mean(residuals))
std_resid = float(np.std(residuals, ddof=1))  # sample std

accuracy = pd.DataFrame({"mean": [mean_resid], "std": [std_resid]})
accuracy.to_csv("backtest-accuracy.tsv", sep="\t", index=False)

accuracy

,mean,std
0,-0.004856,0.274566


Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [8]:
# YOUR CHANGES HERE

seasonal_mean_full = df.groupby("moy")["price"].mean()

future_months = pd.date_range("2025-01-01", "2025-12-01", freq="MS")
forecast_prices = pd.Series(future_months.month).map(seasonal_mean_full).to_numpy()

forecast = pd.DataFrame({
    "month": future_months.strftime("%Y-%m-01"),
    "price": np.round(forecast_prices, 6)
})

forecast.to_csv("strawberry-forecast.tsv", sep="\t", index=False)
forecast.head()

,month,price
0,2025-01-01,4.50120
1,2025-02-01,4.12580
2,2025-03-01,3.69940
3,2025-04-01,3.74496
4,2025-05-01,3.46840


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [9]:
# YOUR CHANGES HERE

BUDGET = 1_000_000.0
storage_cost_per_month = 0.10
sell_discount = 0.10  # 10% discount => receive 90% of sell price

f = forecast.copy()
f["month_dt"] = pd.to_datetime(f["month"])
f = f.sort_values("month_dt").reset_index(drop=True)

rows = []
for i in range(len(f)):
    buy_month = f.loc[i, "month_dt"]
    buy_price = float(f.loc[i, "price"])

    for j in range(i + 1, len(f)):  # must sell after buying
        sell_month = f.loc[j, "month_dt"]
        sell_price = float(f.loc[j, "price"])

        months_held = (sell_month.year - buy_month.year) * 12 + (sell_month.month - buy_month.month)

        # Budget constraint: buy + storage
        per_pint_cost = buy_price + storage_cost_per_month * months_held
        pints = int(math.floor(BUDGET / per_pint_cost))

        # Profit per pint: discounted sell - buy - storage
        per_pint_profit = (1.0 - sell_discount) * sell_price - buy_price - storage_cost_per_month * months_held
        expected_profit = pints * per_pint_profit

        rows.append({
            "buy_month": buy_month.strftime("%Y-%m-01"),
            "sell_month": sell_month.strftime("%Y-%m-01"),
            "pints_purchased": pints,
            "expected_profit": expected_profit
        })

timings = pd.DataFrame(rows)
timings.to_csv("timings.tsv", sep="\t", index=False)

timings.sort_values("expected_profit", ascending=False).head(10)

,buy_month,sell_month,pints_purchased,expected_profit
55,2025-07-01,2025-12-01,271857,174976.408338
50,2025-06-01,2025-12-01,262095,132786.238230
59,2025-08-01,2025-12-01,258986,119349.554324
54,2025-07-01,2025-11-01,279454,107243.267040
62,2025-09-01,2025-12-01,255506,104306.236404
49,2025-06-01,2025-11-01,269150,66415.454000
64,2025-10-01,2025-12-01,244834,58180.882756
58,2025-08-01,2025-11-01,265872,53429.637120
61,2025-09-01,2025-11-01,262205,38900.733800
44,2025-05-01,2025-12-01,239900,36856.796600


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [10]:
# YOUR CHANGES HERE

best_idx = timings["expected_profit"].idxmax()
best_row = timings.loc[best_idx]

best_profit = float(best_row["expected_profit"])

# Per assignment: pints bought in best scenario * backtest std residual
one_std_profit = float(best_row["pints_purchased"] * std_resid)

check = pd.DataFrame({
    "best_profit": [best_profit],
    "one_std_profit": [one_std_profit]
})
check.to_csv("check.tsv", sep="\t", index=False)

check

,best_profit,one_std_profit
0,174976.408338,74642.708223


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


In [11]:
with open("acknowledgments.txt", "w") as f:
    f.write("none\n")

Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.